In [ ]:
!pip install keras-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 31.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import SimpleRNN
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/medium_articles.csv")
print(data)

                                            title  \
0                             Mental Note Vol. 24   
1                       Your Brain On Coronavirus   
2                                  Mind Your Nose   
3                        The 4 Purposes of Dreams   
4                Surviving a Rod Through the Head   
...                                           ...   
192363        Why do you need a cleaning service?   
192364  Daily cleaning and maintenance of bedding   
192365        Beneficial Advice on Bond Cleaning!   
192366    How I Learned Romanian in 37 Easy Steps   
192367     Trying Pimsleur Cantonese in Hong Kong   

                                                     text  \
0       Photo by Josh Riemer on Unsplash\n\nMerry Chri...   
1       Your Brain On Coronavirus\n\nA guide to the cu...   
2       Mind Your Nose\n\nHow smell training can chang...   
3       Passionate about the synergy between science a...   
4       You’ve heard of him, haven’t you? Phineas Gage... 

In [ ]:
from collections import Counter
def pars_tags (data):
  d = {}
  for el in data['tags']:
    for i in el:
      if i in d:
        d[i] += 1
      else:
        d[i] = 1
  cnt = Counter(d).most_common(10)
  return cnt

In [ ]:
def upd_data (data, tag_list):
  new_data = pd.DataFrame()
  for _, el in data.iterrows():
    for i in el['tags']:
      for j in tag_list:
        if i == j[0]:
          el['tags'] = i
          new_data = new_data.append(el, ignore_index = True)
  return new_data  

In [ ]:
tmp = data.tags.apply(lambda x: x[1:-1].split(','))
tmp_data = data.copy() 
tmp_data['tags'] = tmp
tags_list = pars_tags(tmp_data)
new_data = upd_data(tmp_data, tags_list)
print(new_data)

<ipython-input-6-2eb8360fb675>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_data = new_data.append(el, ignore_index = True)


                                                   title  \
0      How to Turn Your Popular Blog Series Into a Be...   
1              Four Exercises to Strengthen Your Writing   
2      This 10-Minute Routine Will Increase Your Clar...   
3              How to Make Your Day Job Support Your Art   
4      An Effective Five-Step Process for Writing Cap...   
...                                                  ...   
50163               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50164               bStable needs you 🦸‍♂️ 🧑‍✈️🧑‍🔬🧑‍💻👨‍🌾   
50165                Status messenger for Desktop review   
50166  The Impact of Technology on Hotel Guest Experi...   
50167  (Live — Streaming) : 2021 Soul Train Music Awards   

                                                    text  \
0      How to Turn Your Popular Blog Series Into a Be...   
1      A few months ago, I wrote an article sharing s...   
2      “Your subconscious mind works continuously, wh...   
3      Gather around kids and let me te

In [ ]:
x = new_data['text']
y = new_data['tags']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
new_data = pd.DataFrame(x)
new_data['tags'] = y
new_data = new_data[:200]
print(new_data)
train, test = train_test_split(new_data, test_size=0.33, random_state=42)

                                                  text  tags
0    How to Turn Your Popular Blog Series Into a Be...     9
1    A few months ago, I wrote an article sharing s...     9
2    “Your subconscious mind works continuously, wh...     4
3    Gather around kids and let me tell you a story...     9
4    You’ve spent hours pouring your heart and soul...     9
..                                                 ...   ...
195  Redefine Publishing\n\nStaying consistent duri...     9
196  The difference between beneficial and destruct...     7
197  Who are Medium’s Curators and What do They do?...     9
198  Section 1: List of Challenges and Related Stor...     9
199  Practical Examples\n\nIterators\n\nIn Python, ...     3

[200 rows x 2 columns]


In [ ]:
x = train["text"]
y = train["tags"]
x = np.asarray(x).astype(np.str_)
y = np.asarray(y).astype(np.float32)
test_x = test["text"]
test_y = test["tags"]
test_x = np.asarray(test_x).astype(np.str_)
test_y = np.asarray(test_y).astype(np.float32)

In [ ]:
# Data
BATCH_SIZE = 64
SEQ_LEN = 128

# Model
EMBED_DIM = 256
FEED_FORWARD_DIM = 256
NUM_HEADS = 3
NUM_LAYERS = 2
VOCAB_SIZE = 10000  # Limits parameters in model.

# Training
EPOCHS = 100

# Inference
NUM_TOKENS_TO_GENERATE = 80

In [ ]:
train = tf.data.Dataset.from_tensor_slices(x)
test = tf.data.Dataset.from_tensor_slices(test_x)

In [ ]:
raw_train_ds = (
    train
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

raw_val_ds = (
    test
    .batch(BATCH_SIZE)
    .shuffle(buffer_size=256)
)

In [ ]:
import keras_nlp
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    raw_train_ds,
    vocabulary_size=VOCAB_SIZE,
    lowercase=True,
    reserved_tokens=["[PAD]", "[UNK]", "[BOS]"],
)

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    sequence_length=SEQ_LEN,
    lowercase=True,
)

In [ ]:
# packer adds a start token
start_packer = keras_nlp.layers.StartEndPacker(
    sequence_length=SEQ_LEN,
    start_value=tokenizer.token_to_id("[BOS]"),
)


def preprocess(inputs):
    outputs = tokenizer(inputs)
    features = start_packer(outputs)
    labels = outputs
    return features, labels


# Tokenize and split into train and label sequences.
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

In [ ]:
inputs = keras.layers.Input(shape=(None,), dtype=tf.int32)
# Embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=VOCAB_SIZE,
    sequence_length=SEQ_LEN,
    embedding_dim=EMBED_DIM,
    mask_zero=True,
)
x = embedding_layer(inputs)
# Transformer decoders.
for _ in range(NUM_LAYERS):
    decoder_layer = keras_nlp.layers.TransformerDecoder(
        num_heads=NUM_HEADS,
        intermediate_dim=FEED_FORWARD_DIM,
    )
    x = decoder_layer(x)  # Giving one argument only skips cross-attention.
# Output.
outputs = keras.layers.Dense(VOCAB_SIZE)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, mask_token_id=0)
model.compile(optimizer="adam", loss=loss_fn, metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        2592768   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 dense (Dense)               (None, None, 10000)       257000

In [ ]:
model.fit(train_ds, validation_data=val_ds, verbose=2, epochs=EPOCHS)

Epoch 1/100
3/3 - 1s - loss: 1.9913 - acc: 0.5802 - val_loss: 6.2361 - val_acc: 0.0982 - 750ms/epoch - 250ms/step
Epoch 2/100
3/3 - 1s - loss: 1.9041 - acc: 0.6129 - val_loss: 6.1979 - val_acc: 0.1051 - 651ms/epoch - 217ms/step
Epoch 3/100
3/3 - 1s - loss: 1.8164 - acc: 0.6188 - val_loss: 6.2579 - val_acc: 0.1007 - 635ms/epoch - 212ms/step
Epoch 4/100
3/3 - 1s - loss: 1.7377 - acc: 0.6425 - val_loss: 6.2618 - val_acc: 0.1038 - 649ms/epoch - 216ms/step
Epoch 5/100
3/3 - 1s - loss: 1.6508 - acc: 0.6561 - val_loss: 6.3256 - val_acc: 0.1024 - 635ms/epoch - 212ms/step
Epoch 6/100
3/3 - 1s - loss: 1.5662 - acc: 0.6804 - val_loss: 6.3253 - val_acc: 0.1047 - 687ms/epoch - 229ms/step
Epoch 7/100
3/3 - 1s - loss: 1.4885 - acc: 0.6915 - val_loss: 6.3847 - val_acc: 0.1029 - 685ms/epoch - 228ms/step
Epoch 8/100
3/3 - 1s - loss: 1.4118 - acc: 0.7160 - val_loss: 6.3997 - val_acc: 0.1074 - 851ms/epoch - 284ms/step
Epoch 9/100
3/3 - 1s - loss: 1.3582 - acc: 0.7197 - val_loss: 6.4825 - val_acc: 0.1016 -

In [ ]:
prompt_tokens = start_packer(tokenizer([""]))
prompt_tokens

<tf.Tensor: shape=(1, 128), dtype=int32, numpy=
array([[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>

In [ ]:
def next(prompt, cache, index):
    logits = model(prompt)[:, index - 1, :]
    # Ignore hidden states for now; only needed for contrastive search.
    hidden_states = None
    return logits, hidden_states, cache

In [ ]:
class TopKTextGenerator(keras.callbacks.Callback):
    """A callback to generate text from a trained model using top-k."""

    def __init__(self, k):
        self.sampler = keras_nlp.samplers.TopKSampler(k)

    def on_epoch_end(self, epoch, logs=None):
        output_tokens = self.sampler(
            next=next,
            prompt=prompt_tokens,
            index=1,
        )
        txt = tokenizer.detokenize(output_tokens)
        print(f"Top-K search generated text: \n{txt}\n")


text_generation_callback = TopKTextGenerator(k=10)
# Dummy training loop to demonstrate callback.
model.fit(train_ds, verbose=2, epochs=2, callbacks=[text_generation_callback])

Epoch 1/2
Top-K search generated text: 
[b'[BOS] what are the benefits of cloud data warehousing ? and should you switch data warehousing was introduced in 1988 by ibm researchers barry devlin and paul murphy . since then the concept has evolved and taken on a life of its own . increasing challenges and complexities of business have forced data warehousing to become a distinct discipline . over the years this has led to best business practices , improved technologies , and h']

3/3 - 7s - loss: 0.0563 - acc: 0.9888 - 7s/epoch - 2s/step
Epoch 2/2
Top-K search generated text: 
[b'[BOS] what story can you tell that only you could ? you have no idea who is waiting on your story this question has bothered me for weeks . the easy deflection i could come up with is that i should just keep writing and the story will tell itself . however , as i study provide value to me : pros aren \xe2\x80\x99 t working . the big page count every easy for so far . then , you hit a wall . your writing becomes 